In [ ]:
# 当前基础环境：py310-torch231-cu121

# 在基础环境之上安装依赖
! pip install swanlab modelscope transformers datasets peft accelerate

In [2]:
from modelscope import MsDataset

# 假设你已经正确加载了 MsDataset 数据集
dataset = MsDataset.load('swift/zh_cls_fudan-news', split='train')
test_dataset = MsDataset.load('swift/zh_cls_fudan-news', subset_name='test', split='test')

print(dataset)
print(test_dataset)

Dataset({
    features: ['text', 'category', 'output'],
    num_rows: 4000
})
Dataset({
    features: ['text', 'category', 'output'],
    num_rows: 959
})


In [2]:
import json

# 转换数据
def construct_instruction_dataset(dataset, output_file):
    messages = []
    
    for data in dataset:  # 遍历加载的数据集
        context = data["text"]      # 获取文本内容
        category = data["category"]  # 获取分类选项
        output = data["output"]         # 获取标签（输出）
        
        # 构造指令格式的数据
        message = {
            "instruction": "作为文本分类专家，请根据给定的文本和分类选项，确定正确的文本类型。",
            "input": f"文本: {context}\n分类选项: {category}",
            "output": output
        }
        messages.append(message)

    # 写入jsonl文件
    with open(output_file, 'w', encoding='utf-8') as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + '\n')

# 执行转换
construct_instruction_dataset(dataset, 'instruction_dataset.jsonl')
construct_instruction_dataset(test_dataset, 'instruction_dataset_test.jsonl')
print("转换完成")

转换完成


In [3]:
# 加载数据集
from datasets import Dataset
import pandas as pd

df = pd.read_json('./instruction_dataset.jsonl', lines=True)
ds = Dataset.from_pandas(df)


In [5]:
# 下载模型
import torch
from modelscope import snapshot_download

model_dir = snapshot_download('qwen/Qwen1.5-7B-Chat', cache_dir='D:/.cache/modelscope/', revision='master')


2024-09-10 04:20:12,273 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2024-09-10 04:23:36,531 - modelscope - WARNING - Downloading: D:/.cache/._____temp\qwen\Qwen1.5-7B-Chat\model-00001-of-00004.safetensors failed, reason: ('Connection broken: IncompleteRead(12332300 bytes read, 155439860 more expected)', IncompleteRead(12332300 bytes read, 155439860 more expected)) will retry
2024-09-10 04:23:48,530 - modelscope - WARNING - Downloading: D:/.cache/._____temp\qwen\Qwen1.5-7B-Chat\model-00001-of-00004.safetensors failed, reason: ('Connection broken: IncompleteRead(21817552 bytes read, 145954608 more expected)', IncompleteRead(21817552 bytes read, 145954608 more expected)) will retry


In [8]:
# 加载模型
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('D:/.cache/modelscope/qwen/Qwen1___5-7B-Chat/', use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained('D:/.cache/modelscope/qwen/Qwen1___5-7B-Chat/', device_map="auto", torch_dtype=torch.bfloat16)
model.enable_input_require_grads()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\miniconda3\envs\test_lora\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\miniconda3\envs\test_lora\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\miniconda3\envs\test_lora\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\miniconda3\envs\test_lora\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\miniconda3\envs\te

In [9]:
# 数据预处理
def preprocess_function(example):
    MAX_LENGTH = 384
    prompt = f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n"
    
    inputs = tokenizer(prompt, add_special_tokens=False)
    outputs = tokenizer(f"{example['output']}", add_special_tokens=False)
    
    input_ids = inputs["input_ids"] + outputs["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = inputs["attention_mask"] + outputs["attention_mask"] + [1]
    labels = [-100] * len(inputs["input_ids"]) + outputs["input_ids"] + [tokenizer.eos_token_id]
    
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenized_dataset = ds.map(preprocess_function, remove_columns=ds.column_names)

Map: 100%|██████████| 4000/4000 [15:07<00:00,  4.41 examples/s]


In [10]:
# 设置lora参数
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [11]:
# 应用lora配置
from peft import get_peft_model

model = get_peft_model(model, lora_config)

In [12]:
# 设置训练参数
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output/Qwen1.5",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [13]:
# 使用swanlab监控
from swanlab.integration.huggingface import SwanLabCallback

swanlab_callback = SwanLabCallback(project="Qwen1.5-Finetune")

In [14]:
# 开始训练
from transformers import Trainer, DataCollatorForSeq2Seq

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

trainer.train()

swanlab: Tracking run with swanlab version 0.3.19                                  
swanlab: Run data will be saved locally in d:\Space\PRO\test\test_lora\swanlog\run-20240910_045709-a3b1799d
swanlab: 👋 Hi samge, welcome to swanlab!
swanlab: Syncing run exp_Sep10_04-57-09 to the cloud
swanlab: 🌟 Run `swanlab watch d:\Space\PRO\test\test_lora\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@samge/Qwen1.5-Finetune
swanlab: 🚀 View run at https://swanlab.cn/@samge/Qwen1.5-Finetune/runs/pe3331lvt83drqrsx40n1


  0%|          | 0/750 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\miniconda3\envs\test_lora\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
d:\miniconda3\envs\test_lora\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:580: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  1%|▏         | 10/750 [01:11<1:23:43,  6.79s/it]

{'loss': 15.2816, 'grad_norm': 0.3484857380390167, 'learning_rate': 9.866666666666668e-05, 'epoch': 0.04}


  3%|▎         | 20/750 [02:19<1:22:34,  6.79s/it]

{'loss': 0.0086, 'grad_norm': 0.011960361152887344, 'learning_rate': 9.733333333333335e-05, 'epoch': 0.08}


...

100%|██████████| 750/750 [1:25:10<00:00,  6.83s/it]

{'loss': 0.045, 'grad_norm': 0.0019128503045067191, 'learning_rate': 0.0, 'epoch': 3.0}


100%|██████████| 750/750 [1:25:11<00:00,  6.82s/it]

swanlab: Step 750 on key train/epoch already exists, ignored.
{'train_runtime': 5121.1254, 'train_samples_per_second': 2.343, 'train_steps_per_second': 0.146, 'train_loss': 1.625669944545254, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=1.625669944545254, metrics={'train_runtime': 5121.1254, 'train_samples_per_second': 2.343, 'train_steps_per_second': 0.146, 'total_flos': 1.96825646628864e+17, 'train_loss': 1.625669944545254, 'epoch': 3.0})

In [25]:
# 模型测试
from peft import PeftModel

lora_path = 'output/Qwen1.5/checkpoint-700'
model = PeftModel.from_pretrained(model, model_id=lora_path)

def predict(text, category_options):
    prompt = f"文本:{text},类型选项:{category_options}"
    messages = [
        {"role": "system", "content": "你是一个文本分类领域的专家，请根据给定的文本和分类选项，确定正确的文本类型。不需要解释，直接输出预测类型，如果没有命中的类型，直接返回：其他。"},
        {"role": "user", "content": prompt}
    ]
    
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([input_text], return_tensors="pt").to('cuda')
    
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=512
        )
    
    response = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    return response.strip()

# 使用示例
test_text = "马斯克：20年内容建立火星基地。"
test_categories = "体育,科技,娱乐,政治"
result = predict(test_text, test_categories)
print(f"预测结果: {result}")

预测结果: 科技
